In [2]:
import wandb
wandb.login(key='local-8c4310dcf3cb371cac7343a466e09de6b04e4282', host='http://localhost:8080')
ENTITY = None

wandb: Currently logged in as: den (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for localhost to your netrc file: /Users/den/.netrc


In [13]:
import ast
import sys
import os
import json
import boto3
from pathlib import Path
from dill.source import getsource
from dill import detect

import pandas as pd
import numpy as np
import plotly
import matplotlib.pyplot as plt

from scipy.stats import ks_2samp
from sklearn import metrics
from sklearn import model_selection
import xgboost as xgb

pd.set_option('display.max_columns', None)

In [14]:
os.environ["AWS_S3_ENDPOINT_URL"] = "http://localhost:8000"
os.environ["AWS_ACCESS_KEY_ID"] = "DevAccessKey"
os.environ["AWS_SECRET_ACCESS_KEY"] = "DevSecretKey"

In [4]:
data_dir = Path('../data/')
model_dir = Path('../models')
model_dir.mkdir(exist_ok=True)

id_vars = ['UniqueID']
targ_var = 'loan_default'
s3_client = boto3.client('s3', endpoint_url="http://localhost:8000",
                         aws_access_key_id="DevAccessKey", aws_secret_access_key="DevSecretKey")


In [5]:
def function_to_string(fn):
    return getsource(detect.code(fn))

In [29]:
run = wandb.init(entity=ENTITY, name="score2", project='credit_scorecard', job_type='preprocess-data', config={'wandb_nb':'wandb_credit_soc'})  # config is optional here


In [30]:
from src.data_utils import (
    describe_data_g_targ,
    one_hot_encode_data,
    create_feature_interaction_constraints,
    get_monotonic_constraints,
    load_training_data,
    calculate_credit_scores
)

from src.scorecard import generate_scorecard

In [31]:
dataset = pd.read_csv(data_dir/'vehicle_loans_subset.csv')
dataset.head()

,UniqueID,LTV,manufacturer_id,Employment_Type,State_ID,PERFORM_CNS_SCORE,AgeInMonths,DaysSinceDisbursement,loan_default
0,417428,88.17,86,Self employed,4,681.0,638.0,153.0,0
1,417429,83.78,86,Salaried,8,384.0,478.0,153.0,0
2,417430,77.39,86,Self employed,6,NaN,441.0,153.0,0
3,417431,72.14,86,Self employed,3,NaN,384.0,153.0,0
4,417432,85.92,86,Self employed,6,721.0,343.0,153.0,0


# additional
p_vars - это фичи, числовые и категориальные
делаем датасет, где подготовлены категориальные колонки

In [32]:
dataset, p_vars = one_hot_encode_data(dataset, id_vars, targ_var)

In [33]:
processed_data_path = data_dir/'proc_ds.csv'
dataset.to_csv(processed_data_path, index=False)

In [52]:
s3_client.upload_file((data_dir/'proc_ds.csv').__str__(), 'creditscore', 'datasets/vehicle_defaults_processed')


In [55]:
os.environ["AWS_S3_ENDPOINT_URL"] = "http://localhost:8000"
os.environ["AWS_ACCESS_KEY_ID"] = "DevAccessKey"
os.environ["AWS_SECRET_ACCESS_KEY"] = "DevSecretKey"
processed_ds_art = wandb.Artifact(name='vehicle_defaults_processed',
                                  type='processed_dataset',
                                  description='One-hot encoded dataset',
                                  metadata={'preprocessing_fn': function_to_string(one_hot_encode_data)}
                                  )
processed_ds_art.add_reference('s3://creditscore/datasets/vehicle_defaults_processed')


[<ManifestEntry ref: s3://creditscore/datasets/vehicle_defaults_processed/vehicle_defaults_processed>]

In [56]:
# Attach our processed data to the Artifact
processed_ds_art.add_file(processed_data_path)

<ManifestEntry digest: cslJubeoq5UrWhqw3sBeqw==>

In [57]:
run.log_artifact(processed_ds_art)
run.finish()

Train dataset size: 0    136907
1     37958
Name: loan_default, dtype: int64 

Validation dataset sizeL 0    45636
1    12653
Name: loan_default, dtype: int64


wandb: Generating checksum for up to 10000 objects with prefix "artifacts"... Done. 0.0s


ValueError: Path is not a file: train.csv

In [62]:
trndat.head()

Number of records (num): 174865
Target count (n_targ): 37958
Target rate (base_rate): 0.21707031138306693
Target odds (base_odds): 0.27725390228403224
Target log odds (base_log_odds): -1.2828215778857626
Dummy model negative log-likelihood (NLL_null): 91484.9725597928
Dummy model LogLoss (LogLoss_null): 0.5231748638080393



,UniqueID,loan_default,AgeInMonths,DaysSinceDisbursement,LTV,PERFORM_CNS_SCORE,Employment_Type__Salaried,Employment_Type__Self employed,State_ID__1,State_ID__2,State_ID__3,State_ID__4,State_ID__5,State_ID__6,State_ID__7,State_ID__8,State_ID__9,State_ID__10,State_ID__11,State_ID__12,State_ID__13,State_ID__14,State_ID__15,State_ID__16,State_ID__17,State_ID__18,State_ID__19,State_ID__20,State_ID__21,State_ID__22,manufacturer_id__45,manufacturer_id__48,manufacturer_id__49,manufacturer_id__51,manufacturer_id__67,manufacturer_id__86,manufacturer_id__120,manufacturer_id__145,manufacturer_id__152,manufacturer_id__153,manufacturer_id__156
17797,435556,0,324.0,141.0,80.00,NaN,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4403,421891,0,351.0,150.0,64.96,718.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
190714,611145,0,255.0,69.0,83.98,NaN,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
115372,534751,0,229.0,98.0,76.38,NaN,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
98591,517772,0,325.0,104.0,77.25,700.0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [63]:
# Create a wandb run, with an optional "log-dataset" job type to keep things tidy
run = wandb.init(entity=ENTITY, project='credit_scorecard', job_type='log-dataset', config={'wandb_nb':'wandb_credit_soc'})  # config is optional here

dataset_art = run.use_artifact('vehicle_defaults_processed:latest', type='processed_dataset')
dataset_dir = dataset_art.download(data_dir)
dataset = pd.read_csv(processed_data_path)

# Set Split Params
test_size = 0.25
random_state = 42

# Log the splilt params
run.config.update({'test_size':test_size, 'random_state': random_state})

# Do the Train/Val Split
trndat, valdat = model_selection.train_test_split(dataset, test_size=test_size,
                                                  random_state=random_state, stratify=dataset[[targ_var]])

print(f'Train dataset size: {trndat[targ_var].value_counts()} \n')
print(f'Validation dataset sizeL {valdat[targ_var].value_counts()}')

# Save split datasets
train_path = data_dir/'train.csv'
val_path = data_dir/'val.csv'
trndat.to_csv(train_path, index=False)
valdat.to_csv(val_path, index=False)

s3_client.upload_file(train_path.__str__(), 'creditscore', 'artifacts/train.csv')
s3_client.upload_file(val_path.__str__(), 'creditscore', 'artifacts/val.csv')

# Create a new artifact for the processed data, including the function that created it, to Artifacts
split_ds_art = wandb.Artifact(name='vehicle_defaults_split',
                              type='train-val-dataset',
                              description='Processed dataset split into train and valiation',
                              metadata={'test_size': test_size, 'random_state': random_state}
                              )
split_ds_art.add_reference('s3://creditscore/artifacts')

# Attach our processed data to the Artifact
# split_ds_art.add_file('train.csv')
# split_ds_art.add_file('val.csv')

# Log the Artifact
run.log_artifact(split_ds_art)

trndict = describe_data_g_targ(trndat, targ_var)

# Create a W&B Table and log 1000 random rows of the dataset to explore
table = wandb.Table(dataframe=trndat.sample(1000))

# Log the Table to your W&B workspace
wandb.log({'processed_dataset': table})

# Close the wandb run
wandb.finish()